In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import datetime as dt
import numpy as np
import altair as alt # 4.1.0

# Snowpark
import snowflake.snowpark as snp

# from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import udf, col, lag, lit, trunc, to_date
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SELECT VARIABLE, DATE, VALUE
FROM cybersyn_financial__economic_essentials.cybersyn.FINANCIAL_FRED_TIMESERIES
LIMIT 10;

In [ ]:

CREATE OR REPLACE TABLE CYBERSYN_FRED_CROSSWALKED AS
SELECT B.SERIES_ID, A.VARIABLE, A.DATE, A.VALUE
FROM cybersyn_financial__economic_essentials.cybersyn.FINANCIAL_FRED_TIMESERIES A 
LEFT JOIN cybersyn_financial__economic_essentials.cybersyn.FINANCIAL_FRED_VARIABLE_SERIES_ID_CROSSWALK B
ON a.VARIABLE = b.VARIABLE
ORDER by DATE;


In [ ]:
SELECT *
FROM CYBERSYN_FRED_CROSSWALKED
WHERE SERIES_ID ilike '%USREC%';

In [ ]:
session.table("CYBERSYN_FRED_CROSSWALKED")


In [ ]:
def macro_df(*fred_symbols):

        CYBERSYN_FRED_CROSSWALKED = session.table("CYBERSYN_FRED_CROSSWALKED")
    
        try:
            fred_symbols[0].split()
        except AttributeError:  # List objects have no split() method.
            fred_symbols = fred_symbols[0]
        # filter to just CPI and =
        final_data = (
            CYBERSYN_FRED_CROSSWALKED
            .filter(CYBERSYN_FRED_CROSSWALKED['SERIES_ID'].in_(fred_symbols))
            .sort(col("DATE"))
            .to_pandas()
            .reset_index()
        )

        return final_data

In [ ]:
inflation_codes = ["CPIAUCSL", "CPILFESL"]

cpi_core_data = macro_df(["CPIAUCSL", "CPILFESL"])

In [ ]:
cpi_core_data.groupby('SERIES_ID').tail(3)

In [ ]:
chart = (
  alt.Chart(cpi_core_data)
  .mark_line()
.encode(x="DATE:T", y="VALUE:Q", color = "SERIES_ID")
)

st.altair_chart(chart, use_container_width=True, theme=None)

In [ ]:
cpi_core_data_yoy = (
    cpi_core_data
    .assign(yoy_change = lambda df: df.groupby("SERIES_ID")['VALUE'].pct_change(12))
)

In [ ]:
cpi_core_data_yoy.groupby("SERIES_ID").tail(2)

In [ ]:
(
    alt.Chart(cpi_core_data_yoy,
        title=alt.Title('CPI v. Core CPI Annual Rates of Inflation',
            fontSize=18,
            subtitle='source: FRED/Cybersyn'))
       .mark_line()
       .encode(
            alt.X('DATE:T').title(None),
            alt.Y('yoy_change:Q').title(None)
                .axis(format='%'),
            alt.Color('SERIES_ID:N').title(None)
                .scale(domain=['CPIAUCSL','CPILFESL'],
                       range=['seagreen', 'darkblue'])
        )
       .properties(width=640, height=480)
   
    + alt.Chart()
        .mark_rule(strokeDash=[5,3])
        .encode(y=alt.datum(0))
)

In [ ]:
unemployment_codes = ["UNRATE"]

unemployment_data = macro_df(unemployment_codes)

unemployment_data.head(5)

In [ ]:
Unemploy = (
    unemployment_data
    .rename(columns={'VALUE': 'UNRATE'})
    .drop(['VARIABLE', 'SERIES_ID'], axis = 1)
)

Unemploy.tail(5)
Unemploy['DATE']  = pd.to_datetime(Unemploy['DATE'])
Unemploy.dtypes

In [ ]:
Sahm = Unemploy.assign(
    UnRate_3m = lambda df: df['UNRATE'].rolling(window=3).mean(),
    UnRate_3m_min = lambda df: df['UnRate_3m'].rolling(window=12).min(),
    Sahm_diff = lambda df: df['UnRate_3m'] - df['UnRate_3m_min']
)

Sahm.tail(3)

In [ ]:
(
alt.Chart(Sahm.query('DATE > "2000-01-01"'),
    title = alt.Title('Sahm Rule Recession Indicator',
        fontSize=18,
        subtitle='source: FRED'))
    .mark_line()
    .encode(
        alt.X('DATE:T').title(None),
        alt.Y('Sahm_diff:Q').title(None)
            .axis(format='%')
    )
    .properties(width=640, height=480)
+
    alt.Chart().mark_rule(color='red', strokeDash=[5,3]).encode(y=alt.datum(0.05))

)

In [ ]:
Rec_dt = macro_df(['USREC'])

Rec_dt.tail(5)

In [ ]:
Rec_DF = pd.DataFrame(
    {
      'start_dt': Rec_dt.query('(USREC==1) & (USREC.shift()==0)')
                        .reset_index()
                        ['DATE'],
      'end_dt': Rec_dt.query('(USREC==0) & (USREC.shift()==1)')
                      .reset_index()
                      ['DATE']
    } )

recession_shade = (
  alt.Chart( Rec_DF )
     .mark_rect(opacity = 0.25, clip=True)
     .encode(
        x = 'start_dt:T',
        x2 = 'end_dt:T'
     )
)